This file takes feature vector as input. 
populates the data using sparse matrix.
shuffles data rows for future cross-validation
store data in a .csv file (because re-running this file again and again was time consuming)

In [42]:
# importing stuff
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
plt.rcParams['figure.figsize']=(10,6)
import glob
from string import punctuation
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
import operator
from scipy.sparse import csr_matrix
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False


In [43]:
# merge frequencies
def getDictOfWords(path):
    words_list = parse_text_tolist(path)
    freq = freqs(words_list)

    # merging all values for a key and forming a new dict of key value pairs
    from collections import defaultdict
    freqs_final = defaultdict(list)
    for k,v in freq.items():
        freqs_final[k].append(v)


    freqs_final = {k: sum_list(v) for k, v in freqs_final.items()}
    return freqs_final

In [44]:
# get location of nonzero element while populating data
def getDatapoint(dict_of_features, dict_of_newfile):
    d_feature_list = dict_of_features
    d_inputfile = dict_of_newfile

    row_loc = []
    row_val = []
    it = 0
    for key, value in d_feature_list.items():
        if key in d_inputfile:
            row_loc.append(it)
            d_feature_list[key] = d_inputfile[key]
            row_val.append(d_feature_list[key])
        else:
            d_feature_list[key] = 0
        it = it + 1
    return row_loc, row_val

In [45]:
# function to read the file and return a list of words
def parse_text_tolist(path):
    try:
        soup=open(path,'r')
        raw_text=strip_pnctn(soup)
        word_list = raw_text.split()
        words_list=[]
        for i in range(len(word_list)):
            chars = ''.join(s for s in word_list[i] if s.isalnum()) #splitting into chars for aplhanumeric
            if is_number(chars)==False: #removing numbers
                words_list.append(chars)
    except ValueError:
        print("value Error")
    words_list= [x.lower() for x in words_list]
    return words_list

In [46]:
# counting the word frequency
def freqs(wl):
    wordfreq = []
    wordname=[]
    for w in set(wl):
        wordfreq.append(wl.count(w))
        wordname.append(w)
    freq=dict(zip(wordname,wordfreq))
    return freq

In [47]:
# sum all elements in a list
def sum_list(values):
    v= sum(values)
    return v

In [48]:
# function to parse the text ignoring the chars
def strip_pnctn(txt):
    return ','.join(c for c in txt if c not in ignoreList)

In [49]:
# Include stop-words
def getStopWords():
    with open('C:\\Users\\athar\\Downloads\\Machine Learning-20170419T225814Z-001\\Machine Learning\\terrier-stop.txt', 'r') as f:
        stopwords1 = [x.strip('\n') for x in f.readlines()]

    with open('C:\\Users\\athar\\Downloads\\Machine Learning-20170419T225814Z-001\\Machine Learning\\minimal-stop.txt', 'r') as f:
        stopwords2 = [x.strip('\n') for x in f.readlines()]

    with open('C:\\Users\\athar\\Downloads\\Machine Learning-20170419T225814Z-001\\Machine Learning\\stoplist.txt', 'r') as f:
        stopwords3 = [x.strip('\n') for x in f.readlines()]

    stopwords4 = stopwords.words('english')

    stpwrd_full = stopwords1 +stopwords2 +stopwords3+stopwords4
    
    # List of problem specific stopwords
    mylist_stopwords = ["newsgroups","messageid",'gmt','path','subject','sender','lines','organization','articleid','apr','date','writes','pp','cantaloupesrvcscmuedu','xref','nntppostinghost','article','references']
#    common_keys= list(set(keys)&set(stpwrd_full)) + mylist_stopwords #mega keywords list
    common_keys= list(set(stpwrd_full)) + mylist_stopwords #mega keywords list 
    return common_keys

Load features vector

In [50]:
# Read already obtained features list
fl=pd.read_csv("C:\\Users\\athar\\Downloads\\ML_project\\Features_List_30.csv")
fl=fl[[1,2]]
fl.head()

,0,1
0,time,180780
1,system,164799
2,god,152213
3,file,112007
4,government,103203


In [51]:
fl=fl.as_matrix()

In [52]:
d_mega_features={x[0]:x[1] for x in fl}
len(d_mega_features) # check number of features

1094

In [58]:
row_count = 0
row_nonzero = []
col_nonzero = []
nonzero_elements = []
Ytrain = []

###global part of the code
common_keys = getStopWords()
# creating a list of chars to be ignored
num=['0','1','2','3','4','5','6','7','8','9','--']
ignoreList= punctuation + "1 2 3 4 5 6 7 8 9 0"


In [53]:
li=['alt.atheism','talk.religion.misc','soc.religion.christian', 'talk.politics.guns',\
   'talk.politics.mideast', 'talk.politics.misc', 'sci.space', 'sci.med', 'sci.electronics',\
   'sci.crypt', 'rec.sport.baseball', 'rec.motorcycles', 'rec.autos', 'rec.sport.hockey',\
   'misc.forsale','comp.windows.x','comp.sys.mac.hardware','comp.sys.ibm.pc.hardware',\
   'comp.os.ms-windows.misc','comp.graphics']

Load all data points and their respective classes

In [61]:
classNo = 1
Y_for_cv = []
row_count = 0
for l in li:
    pl="C:\\Users\\athar\\Downloads\\ML_project\\20_newsgroups\\"+l+"\\*"
    path_list=glob.glob(pl)
    path_list_100= path_list[:1000] #take the first 100 files 
    
    for file in path_list_100:
        # this will get a dict of words from a given file at location path
        freqs_final = getDictOfWords(file)
        col_nonzero1, val_nonzero = getDatapoint(d_mega_features, freqs_final)

        # Build elements of sparse matrix
        row_nonzero.extend(np.ones(np.size(col_nonzero1), dtype = np.int) * row_count)
        col_nonzero.extend(col_nonzero1)
        nonzero_elements.extend(val_nonzero)

        row_count = row_count + 1
        Y_for_cv.append(classNo)
    
    classNo+=1

In [62]:
# combine elements defined above to form a sparse matrix
X_for_cv = csr_matrix((nonzero_elements, (row_nonzero, col_nonzero)), shape=(row_count, len(d_mega_features)))#.toarray()

In [63]:
# Putting data into a dataframe
X_CV=pd.DataFrame(X_for_cv.toarray())
X_CV.head()

,0,1,2,3,4,5,6,7,8,9,...,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093
0,0,1,13,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,0,55,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
Y_CV=pd.DataFrame(Y_for_cv)
Y_CV.columns=['y']
Y_CV.head()

,y
0,1
1,1
2,1
3,1
4,1


In [65]:
result1 = pd.concat([X_CV, Y_CV], axis=1)
result1.head()

,0,1,2,3,4,5,6,7,8,9,...,1085,1086,1087,1088,1089,1090,1091,1092,1093,y
0,0,1,13,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,7,0,55,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [66]:
# re-shuffle the rows in available data for stratification 
result1 = result1.sample(frac=1).reset_index(drop=True)
result1.head()

,0,1,2,3,4,5,6,7,8,9,...,1085,1086,1087,1088,1089,1090,1091,1092,1093,y
0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,15
1,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,18
2,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,12
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,2,1,1,0,0,0,1,0,1,0,...,0,0,0,0,1,0,0,0,0,7


Store entire data to csv

In [67]:
result1.to_csv("C:\\Users\\athar\\Downloads\\ML_project\\dataset_30f.csv")